In [ ]:
!apt install peg
%pip install spleeter


In [ ]:
%pip install xformers==0.0.19 torch==2.0.0+cu118 torchvision  torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
%pip install pytube moviepy spleeter
%pip uninstall ffmpeg-python -y
%pip install ffmpeg-python


In [ ]:
%pip install speechbrain soundfile


In [ ]:
%pip install pydub 

In [ ]:
!mkdir  /notebooks/motivation
%cd /notebooks/motivation

In [ ]:
from pytube import YouTube

# specify the URL of the video
url = 'https://www.youtube.com/watch?v=GEyn-nzqiNo'
yt = YouTube(url)

# download the highest resolution version of the video
ys = yt.streams.get_highest_resolution()
ys.download()


In [ ]:
%pip install git+https://github.com/espnet/espnet
%pip install -q espnet_model_zoo

In [ ]:
from email.mime import audio
from moviepy.editor import *
import os


def extract_audio(video_path, audio_output_path):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_output_path)
    
videos_directory = '/notebooks/motivation'
audio_files_directory = './audio'
audio_enhanced_directory = './audio_enhanced'
audio_separated_directory = './audio_separated'
audio_separated_vocals_directory = './audio_separated_vocals'


In [ ]:
%cd {videos_directory}

In [ ]:
if not os.path.exists(audio_enhanced_directory):
    os.makedirs(audio_enhanced_directory)

In [ ]:
if not os.path.exists(audio_files_directory):
    os.makedirs(audio_files_directory)

for video_file in os.listdir(videos_directory):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(videos_directory, video_file)
        audio_output_path = os.path.join(audio_files_directory, f'{video_file[:-4]}.wav')
        extract_audio(video_path, audio_output_path)

In [ ]:
import moviepy.editor as mp
import os
def split_audio(file_path, output_dir):
    audio = mp.AudioFileClip(file_path)
    duration = audio.duration
    intervals = int(duration // 300) + (1 if duration % 300 > 0 else 0)
    for i in range(intervals):
        start_time = i * 300
        end_time = (i + 1) * 300 if (i + 1) * 300 < duration else duration
        chunk = audio.subclip(start_time, end_time)
        chunk_filename = os.path.join(output_dir, f"{i}_{os.path.basename(file_path)}")
        chunk.write_audiofile(chunk_filename)
    audio.close()

In [ ]:
input_dir = audio_files_directory
audio_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp3', '.wav'))]


In [ ]:
output_dir = "./audio_chunks"


In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for audio_file in audio_files:
    file_path = os.path.join(input_dir, audio_file)
    split_audio(file_path, output_dir)

In [ ]:
import os

import numpy as np
from pydub import AudioSegment
import soundfile as sf

input_dir = "./audio_chunks"
output_dir = "./audio_enhanced_chunks"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
audio_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp3', '.wav'))]
for audio_file in audio_files:
    file_path = os.path.join(input_dir, audio_file)

    # Load audio file using pydub
    mixwav_mc, sr = soundfile.read(file_path)
    mixwav_sc = mixwav_mc[:,1]
    wave = enh_model_sc(mixwav_sc[None, ...], sr)

    # Create a new filename for the enhanced audio
    enhanced_filename = f"enhanced_{audio_file}"
    enhanced_filepath = os.path.join(output_dir, enhanced_filename)

    # Save the enhanced audio to the output_dir
    sf.write(enhanced_filepath, wave[0].squeeze(), sr)

In [ ]:
from pydub import AudioSegment

output_dir = "./audio_enhanced_chunks"
combined_output_file = "./combined_enhanced_audio.wav"

# Get a list of all the enhanced audio files
enhanced_audio_files = [f for f in os.listdir(output_dir) if f.endswith(('.wav', '.mp3'))]

# Initialize an empty audio segment
combined_segment = AudioSegment.empty()

# Iterate through each enhanced audio file, loading and concatenating them
for enhanced_audio_file in enhanced_audio_files:
    enhanced_filepath = os.path.join(output_dir, enhanced_audio_file)
    enhanced_segment = AudioSegment.from_wav(enhanced_filepath) if enhanced_audio_file.endswith('.wav') else AudioSegment.from_mp3(enhanced_filepath)
    combined_segment += enhanced_segment

# Export the combined audio segment to a new file
combined_segment.export(combined_output_file, format="wav")

In [ ]:
%pip install pyannote.audio

In [ ]:
import os
audio_file="/notebooks/tamioc2/audio/vocals/vocal_Tamioc, le nouveau podcast du Pacifique.wav_10.wav"
if not os.path.exists(audio_file):
    os.makedirs(audio_file)

In [ ]:
from pyannote.audio import Pipeline
import torch
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.0",
                                    use_auth_token="")

pipeline.to(torch.device("cuda"))


In [ ]:
import torchaudio
from pyannote.audio.pipelines.utils.hook import ProgressHook
with ProgressHook() as hook:
    waveform, sample_rate = torchaudio.load(audio_file)
    diarization = pipeline({"waveform": waveform, "sample_rate": sample_rate}, hook=hook)



In [ ]:
from pyannote.core import Segment
from pyannote.audio import Audio
from pydub import AudioSegment

output_dir = "./by_speaker"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# Load the audio file
audio = AudioSegment.from_wav(audio_file)

for turn, _, speaker in diarization.itertracks(yield_label=True):
    # Get the start and end times (in milliseconds) of the speaker segment
    start_time_ms = int(turn.start * 1000)
    end_time_ms = int(turn.end * 1000)

    # Extract the speaker segment from the audio file
    speaker_segment = audio[start_time_ms:end_time_ms]

    # Build the output filename
    speaker_label = f'speaker_{speaker}_'
    segment_label = f'{turn.start:.3f}-{turn.end:.3f}'.replace('.', '_')
    filename = f'{speaker_label}{segment_label}.wav'

    # Save the speaker segment to disk
    output_path = os.path.join(output_dir, filename)
    speaker_segment.export(output_path, format="wav")

In [ ]:
import os
import shutil
output_dir = "./by_speaker"
source_directory = output_dir # replace with the path to your directory
files = [f for f in os.listdir(source_directory) if os.path.isfile(os.path.join(source_directory, f))]
for file in files:
    # Extract the speaker's name from the filename
    speaker_name = file.split('_')[2]
    
    # Create a new directory for the speaker if it doesn't already exist
    speaker_directory = os.path.join(source_directory, speaker_name)
    if not os.path.exists(speaker_directory):
        os.makedirs(speaker_directory)
    
    # Move the file into the speaker's directory
    source_file_path = os.path.join(source_directory, file)
    destination_file_path = os.path.join(speaker_directory, file)
    shutil.move(source_file_path, destination_file_path)

In [ ]:
output_dir = "./by_speaker"
input_dir = output_dir
output_dir = "tamioc_merged_by_speaker"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
for speaker in set(speaker for filename in os.listdir(input_dir) for speaker in [filename.split('_')[2]]):
    # Initialize an empty audio segment
    merged_clip = AudioSegment.empty()
    # List to hold clips temporarily
    temp_clips = []
    # Counter for naming output files
    counter = 0

    # Get a list of all clips for this speaker
    speaker_clips = [filename for filename in os.listdir(input_dir) if filename.startswith("speaker_SPEAKER_"+speaker) and os.path.isfile(os.path.join(input_dir, filename))]
    
    for filename in speaker_clips:
        # Load the clip
        clip = AudioSegment.from_wav(os.path.join(input_dir, filename))
        temp_clips.append(clip)
        # Check the total duration of clips in temp_clips
        total_duration_ms = sum(temp_clip.duration_seconds for temp_clip in temp_clips) * 1000
        
        # If total duration reaches 5 minutes, merge and save the clips
        if total_duration_ms >= 300000:
            merged_clip = sum(temp_clips, AudioSegment.empty())
            output_filename = f"{speaker}_{counter:03d}.wav"
            merged_clip.export(os.path.join(output_dir, output_filename), format="wav")
            # Reset temp_clips and increment counter
            temp_clips = []
            counter += 1

    # Merge and save any remaining clips
    if temp_clips:
        merged_clip = sum(temp_clips, AudioSegment.empty())
        output_filename = f"{speaker}_{counter:03d}.wav"
        merged_clip.export(os.path.join(output_dir, output_filename), format="wav")